In [1]:
    import abc, fnmatch, io, dataclasses, IPython, jinja2, inspect, collections, mistune, toolz.curried as toolz, base64, string, traitlets
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    try: 
        from .utils import format_images
    except: 
        from utils import format_images
    ip = IPython.get_ipython()

In [2]:
def strip_graphviz_headers(Source):
        str = Source._repr_svg_()
        # Mistune does a bad job of handling inline comments.
        for i in range(4): str = str.partition('>')[2]
            
        # Mistune has a hard time with the xmlns information.
        for year in (1999, 2000): str = str.replace(F'xmlns:xlink="http://www.w3.org/{year}/xlink"', '')
        return str.lstrip()

In [ ]:
def graphviz(ip, str): 
    try: import graphviz
    except: return
    if '->' in str and not str.startswith('di'): str = 'di' + str
    return IPython.display.HTML(strip_graphviz_headers(graphviz.Source(str)))

In [ ]:
def eval_shorthand_ipython(ip, str):
    return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [ ]:
    def flatten(str): return ''.join(str.splitlines())

In [ ]:
def embed(ip, str):
    type = guess(str) or ''
    if str.startswith('http'): 
        if type.startswith('image') and not type.endswith('svg'):
            return F"""<img src="{str}"/>"""
        return F"""<iframe src="{str}" width="100%" height="500px"></iframe>"""

In [ ]:
    mimes = toolz.pipe("""
        50 text/html
        70 text/latex
        80 image/svg+xml
        90 image/bmp image/png image/jpeg image/gif'
        60 text/markdown
        110 text/javascript application/javascript 
        120 text/plain
        """, str.strip, str.splitlines, toolz.map(toolz.compose(toolz.juxt(toolz.compose(int, next), list), iter, str.split)), list)

In [ ]:
  def ipython_formatter(object):
        bundle, metadata = IPython.get_ipython().display_formatter.format(object)
        for type in toolz.pipe(mimes, toolz.map(toolz.second), toolz.concat):
            if type == 'text/plain': return object
            if type in bundle:  
                object = bundle[type]
                if type.startswith('image') and 'svg' not in type: 
                    object = format_images(type, bundle)._repr_html_()
                if type == 'text/html':
                    object = flatten(object)
                break
        return object

In [ ]:
class StringFormatter(IPython.core.formatters.MimeBundleFormatter):
    mimebundle_formatter = traitlets.Instance(IPython.core.formatters.MimeBundleFormatter)
    formatters = traitlets.Dict()
    environment = traitlets.Instance(jinja2.Environment, kw={
        'finalize': ipython_formatter,
        'lstrip_blocks': True
    })
    level = traitlets.Int(default_value=0)
    @traitlets.default('formatters')
    def _default_formatters(self):return {
        'https://*' : embed,
        'http://*' : embed,
        'graph {*}': graphviz, 
        'digraph {*}': graphviz,
        ';[! ]*': eval_shorthand_ipython,
        ',[! ]*': eval_shorthand_ipython,
    }
    
    def __enter__(self): self.level += 1
    def __exit__(self, *exc): self.level -= 1
    
    def __call__(self, object, include=None, exclude=None, display=False):
        if isinstance(object, str):
            with self:
                for type, callable in self.formatters.items():
                    if fnmatch.fnmatch(object, type):
                        result = callable(self.parent, object)
                        if result: return ip.display_formatter.format(result)
                if self.level == 1: return self.format_html(object)
        return self.mimebundle_formatter(object, include, exclude)
    
    def format_html(self, object):
        return {
            'text/html': self.environment.from_string(
                mistune.markdown(object, escape=False, parse_block_html=True, parse_inline_html=True)
            ).render(**self.parent.user_ns), 'text/markdown': object, 'text/plain': object
        }, {}

IPython.core.formatters.FormatterABC.register(StringFormatter);

In [ ]:
def format_module_doc(type):
    if type.__doc__:
        return IPython.get_ipython().display_formatter.format(type.__doc__)

In [11]:
   def show_axes(object):
        ip = IPython.get_ipython()
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(object.figure).print_png(bytes)
        bundle = {
            'image/png': bytes.getvalue()
        }
        return {
            'text/html': format_images('image/png', bundle)._repr_html_(),
            'text/plain': repr(object),
        }, {}

In [12]:
def load_ipython_extension(ip): 
    ip = ip or IPython.get_ipython()
    import matplotlib.figure

    ip.display_formatter.mimebundle_formatter.for_type(matplotlib.figure.Axes, show_axes)
    
    ip.display_formatter.mimebundle_formatter = StringFormatter(
        mimebundle_formatter=ip.display_formatter.mimebundle_formatter, parent=ip)
    ip.display_formatter.mimebundle_formatter.for_type(type, format_module_doc)
    
def unload_ipython_extension(ip=None):
    ip.display_formatter.mimebundle_formatter = IPython.core.formatters.MimeBundleFormatter(parent=ip)
    
__name__ == '__main__' and load_ipython_extension(get_ipython())